In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import time
import datetime

In [32]:
flights_test_2020 = pd.read_csv('data/flights_test_2020.csv')
df = pd.read_csv('data/flights_test.csv')
p_df = pd.read_csv('data/Merged_Table_1.csv')
flights_test_2020.drop(columns='Unnamed: 0',inplace=True)

In [33]:
df.head(5)

,date_format,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,1577854800000,WN,WN,WN,5888,WN,N951WN,5888,13891,...,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01,1577854800000,WN,WN,WN,6276,WN,N467WN,6276,13891,...,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01,1577854800000,WN,WN,WN,4598,WN,N7885A,4598,13891,...,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01,1577854800000,WN,WN,WN,4761,WN,N551WN,4761,13891,...,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01,1577854800000,WN,WN,WN,5162,WN,N968WN,5162,13891,...,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


In [56]:
flights_test_2020.shape, flights_test_2020.columns

((150623, 17),
 Index(['mkt_unique_carrier', 'origin_city_name', 'dest_city_name',
        'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance',
        'origin_city_condn', 'dest_city_cond', 'day', 'day_Week',
        'Trip_count_PerDay', 'air_time', 'actual_elapsed_time',
        'mkt_carrier_fl_num', 'Minutes_to_wheels_off',
        'Minutes_from_wheels_on'],
       dtype='object'))

In [57]:
flights_test_2020.isnull().sum()

mkt_unique_carrier        0
origin_city_name          0
dest_city_name            0
crs_dep_time              0
crs_arr_time              0
crs_elapsed_time          0
distance                  0
origin_city_condn         0
dest_city_cond            0
day                       0
day_Week                  0
Trip_count_PerDay         0
air_time                  0
actual_elapsed_time       0
mkt_carrier_fl_num        0
Minutes_to_wheels_off     0
Minutes_from_wheels_on    0
dtype: int64

In [36]:
#extract day and dayofweek
flights_test_2020['day'] = pd.DatetimeIndex(flights_test_2020['fl_date']).day
flights_test_2020['day_Week'] = pd.DatetimeIndex(flights_test_2020['fl_date']).dayofweek
flights_test_2020.drop(columns='fl_date',inplace=True)

In [37]:
def TripcountPerDay(df):
    """function to extract trip count per day per routes origin to dest city"""
    import pandas as pd
    df = df.groupby(['origin_city_name','dest_city_name'])['Trip_count_PerDay'].mean().round(0).astype(int)
    return pd.DataFrame(df.reset_index())

In [38]:
tripcountperday = TripcountPerDay(p_df)

In [39]:
#merge the trip count per day dataframe to flight_test data
flights_test_2020 = pd.merge(flights_test_2020, tripcountperday,how='left', on=['origin_city_name','dest_city_name'])

In [40]:
#fill na tripcount perday with mean
flights_test_2020['Trip_count_PerDay'].fillna(flights_test_2020['Trip_count_PerDay'].mean(), inplace=True)
flights_test_2020['Trip_count_PerDay'] = flights_test_2020['Trip_count_PerDay'].round(0).astype(int)

In [41]:
#extract features to predict air_time, wheels_off and actual_elapse_time
X_woff = flights_test_2020[['crs_dep_time']]
X_air_time = flights_test_2020[['distance']]
X_actual_elapsed = flights_test_2020[['distance','crs_elapsed_time']]

In [42]:
#load the model from pickle
import pickle
wheels_off = pickle.load(open('features prediction/pred_wheels_off_model.sav', 'rb'))
air_time = pickle.load(open('features prediction/air_time_pred_model.sav', 'rb'))
actual_elapse_time = pickle.load(open('features prediction/pred_actual_elapsed_time_model.sav', 'rb'))

pred_wheels_off = wheels_off.predict(X_woff)
pred_air_time = air_time.predict(X_air_time)
pred_actual_elapsed = actual_elapse_time.predict(X_actual_elapsed)

In [59]:
flights_test_2020['wheels_off'] = pred_wheels_off.round(0).astype(int)
flights_test_2020['air_time'] = pred_air_time.round(0).astype(int)
flights_test_2020['actual_elapsed_time'] = pred_actual_elapsed.round(0).astype(int)
flights_test_2020['mkt_carrier_fl_num'] = df['mkt_carrier_fl_num']
flights_test_2020.rename(columns={'dest_city_cond': 'dest_city_condn'},inplace=True) 

In [50]:
#format wheels_off and air_time
from function_helpers import format_air_time
from function_helpers import to_int_hour
flights_test_2020['wheels_off'] = flights_test_2020['wheels_off'].apply(to_int_hour)
flights_test_2020['air_time'] = flights_test_2020['air_time'].apply(format_air_time)

#calculate the wheels_on (wheels_off + air_time)
flights_test_2020['wheels_on'] = flights_test_2020['wheels_off'].astype(int) + flights_test_2020['air_time'].astype(int)
flights_test_2020['wheels_on'] = flights_test_2020['wheels_on'].apply(to_int_hour)

#calcualte the 'Minutes_to_wheels_off', 'Minutes_from_wheels_on'
flights_test_2020['Minutes_to_wheels_off'] = flights_test_2020['wheels_off'].astype(int) - flights_test_2020['crs_dep_time'].astype(int)*100
flights_test_2020['Minutes_from_wheels_on'] = flights_test_2020['wheels_on'].astype(int) - flights_test_2020['crs_arr_time'].astype(int)*100

#drop wheels_off, wheels_on
flights_test_2020.drop(columns=['wheels_off','wheels_on'],inplace=True)

In [79]:
flights_test_2020.head(5)

,mkt_unique_carrier,mkt_carrier_fl_num,origin_city_name,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,actual_elapsed_time,air_time,distance,Trip_count_PerDay,origin_city_condn,dest_city_condn,Minutes_to_wheels_off,Minutes_from_wheels_on,day,day_Week
0,9,5888,239,290,18,19,95,91,102,363,6,7,7,18,20,1,2
1,9,6276,239,290,11,13,90,87,102,363,6,7,7,108,10,1,2
2,9,4598,239,291,20,21,70,70,058,333,11,7,1,4,2,1,2
3,9,4761,239,291,13,14,75,74,058,333,11,7,1,54,12,1,2
4,9,5162,239,291,9,10,80,78,058,333,11,7,1,122,120,1,2


In [61]:
#label encoder the mkt_unique_carrier,origin_city_name,dest_city_name,origin_city_condn,dest_city_cond
from function_helpers import MultiColumnLabelEncoder

cols = ['mkt_unique_carrier','origin_city_name','dest_city_name','origin_city_condn','dest_city_condn']
multi = MultiColumnLabelEncoder(columns=cols)

flights_test_2020 = multi.fit_transform(flights_test_2020)

In [82]:
#load pickle to predict arrial delay
arr_delay = pickle.load(open('Models_predict_arr_delay/pred_arr_delay_model.sav', 'rb'))
wheels_off.feature_names_in_

array(['mkt_unique_carrier', 'mkt_carrier_fl_num', 'origin_city_name',
       'dest_city_name', 'crs_dep_time', 'crs_arr_time',
       'crs_elapsed_time', 'actual_elapsed_time', 'air_time', 'distance',
       'Trip_count_PerDay', 'origin_city_condn', 'dest_city_condn',
       'Minutes_to_wheels_off', 'Minutes_from_wheels_on', 'day',
       'day_Week'], dtype=object)

In [83]:
columns = wheels_off.feature_names_in_.tolist()
flights_test_2020 = flights_test_2020[columns]

In [84]:
#make sure the columns in order as pickle models
flights_test_2020.columns

Index(['mkt_unique_carrier', 'mkt_carrier_fl_num', 'origin_city_name',
       'dest_city_name', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'distance', 'Trip_count_PerDay',
       'origin_city_condn', 'dest_city_condn', 'Minutes_to_wheels_off',
       'Minutes_from_wheels_on', 'day', 'day_Week'],
      dtype='object')

In [88]:
#predict arr_delay
arr_delay_preds = arr_delay.predict(flights_test_2020)
arr_delay_preds

array([27.30727376, 18.69091064,  7.89      , ..., 43.16545532,
       35.43      , 10.02455604])

In [89]:
# reverse the labelencoder, get back informations
flights_test_2020 = multi.inverse_transform(flights_test_2020)

In [93]:
flights_test_2020['arr_delay'] = arr_delay_preds.round(3)

In [95]:
flights_test_2020.to_csv('flights_2020_pred_arr_delay.csv')